# <strong><u>Knapsack Problem</u></strong>

# Introduction


<p>The objective of this project is to implement some algorithms for solving the Knapsack Problem and to provide an experimental study of their running time and quality.
The algorithm implemented in this document are the followings: 
<ul>
    <li> The <a href="#generator">brute-force</a> algorithm that explore all the possible solutions in a systematic way,</li>
    <li> Multiple branch and bound version </li>
    <ul>
        <li> DFS </li>
    </ul>
...
</ul> 

# <h1 id="main">Main</h1>


In [ ]:
############## IMPORT ##############
import sys
import time
import math
import copy
import random
import itertools
import numpy as np
import tracemalloc
from decimal import *
from tabnanny import check
from scipy.stats import beta
from operator import itemgetter
import matplotlib.pyplot as plt
from prettytable import PrettyTable

############## CONSTANT ##############

        ####### ALGO NAME #######
# These constant will be used to display the solution
BRUTE_FORCE = "BRUTE FORCE"

DYNAMIC_PROG = "DYNAMIC PROG"
DYNAMIC_PROG_TOP_DOWN = "DYNAMIC PROG TOP DOWN"
FPTAS = "FPTAS"

GREEDY_FRACTIONAL = "GREEDY FRACTIONAL"
GREEDY_WEIGHT = "GREEDY WEIGHT"
GREEDY_VALUE = "GREEDY VALUE"
GREEDY_RATIO = "GREEDY RATIO"

BRANCH_BOUND = "BRANCH BOUND"

GENETIC = "GENETIC"

RANDOMIZED = "RANDOMIZED"

GENETIC_NO_OPTIMA = "GENETIC NO OPTIMA"
GENETIC_OPTIMA = "GENETIC_OPTIMA"

########################################

##### Global arrays #####

# Save the solutions finded by an algorithm
problem_solutions = []

# Save the items generated by one of the generator
choosed_items = []

# Save the optimal solution readed from the dataset
optimal = [] # must be filled by 0 and 1

# Fixing random state for reproducibility
np.random.seed(19680801)

# increase max recursion for long stack
sys.setrecursionlimit(150000)

## 1. CLASS

<p>In this section we will reference all the class used in the program </p>

### 1. Item

<p>The Item class, is the class that represent an object that we will put in the knapsack, thus each item have a weight and value.
We will differentiate the items by their "id" (their position), inside the array named "choosed_items"</p>

In [ ]:
class Item:
  def __init__(self, value, weight):
    self.value = value
    self.weight = weight
  
  # function that display an item weight and value
  def print(item):
    print("value = " + str(item.value) + " - weight = " + str(item.weight), end=' ')

  #function that display a set of items  
  def print_items(items): 
    # create a table with the column title
    table = PrettyTable(["Item", "Weight", "Value"])
    
    # for each item add the value of the item inside the right row
    for i in range (0, len(items)):
      table.add_row([str(i), str(items[i].weight), items[i].value])
    
    # display the table
    print(table)


#### 1. Total Weight

In [ ]:
# Compute the total weight of set of items
def compute_total_weight(items):
    total_weight = 0
    
    # for each item add the weight of the item to total_weight
    for i in range(0, len(items)):
        total_weight += items[i].weight
    
    return total_weight

#### 2. Total Value

In [ ]:
# Compute the total value of set of items
def compute_total_value(items):
    total_value = 0

    # for each item add the weight of the item to total_weight
    for i in range(0, len(items)):
        total_value += items[i].value

    return total_value

### 1. Solution

<p>The Solution class, is the class that will store the solution computed by the alogorithm, this class will store the name of the algorithm that have solved the problem, the solution founded and some data about the memory usage during the computation of the algorithm. We will solve a probleme multiple time with different algorithm, that is why we created an array, called "problem_solutions" that will store the Solutions object.</p>

In [ ]:
class Solution:
  def __init__(self, time, name, solution, mem_usage, mem_peak):
    self.name = name
    self.time = time
    self.solution = solution
    self.mem_usage = mem_usage
    self.mem_peak = mem_peak
  
  def print(solution):
    print(solution.name + "\t\t - time = " + str(solution.time) + "\t - solution = " + str(solution.solution), end=' ')

  def print_solutions(solutions):
    for i in range(0, len(solutions)):
      print(str(i) + " : ")
      Solution.print(solutions[i])
      print()

#### 1. Solution from genome

<p>Sometimes we will want to decipher a genome (we call genome an encoded solution with 0 and 1), the function below will take a genome decipher it and return the solution associated.</p>

In [ ]:
# compute the solution from a genome
def compute_genome_solution(genome):
    solution = 0

    for i in range(0, len(genome)):
        # when an item is a solution we add his value to the total solution
        if genome[i] == 1:
            solution += choosed_items[i].value
    
    return solution

## 2. SOURCE DATA

<p>There is two possibilities to test the algorithm. The data can be provided by the user from the terminal or the user can load the data from a dataset file that are in the dataset folder. </p>

> **NOTE**\
> If you load data from the dataset you musn't launch the generators.

### 1. input

<p>By launching this block you will be asked to give the number of items you want to generate and the maximal capacity of the knapsack</p>

In [ ]:
# Get the number of item from terminal
nb_item = int(input("Enter number of item  - max :"))

# Get the capacity of the knapsack from terminal
capacity = int(input('Enter the capacity of the sac : '))

# clear the choosed items
choosed_items = []

### 2. Dataset

<p>By launching this block the program will retrieve data from the file provided, fill the "choosed_items" array and set the number of the maximal capacity of the knapsack. The "large_scale" dataset give the optimal solution encoded in 0 and 1, so we will also store the optimal solution int the "optimal" array</p>

In [ ]:
# source of the dataset : http://artemisa.unicauca.edu.co/~johnyortega/instances_01_KP/

# clear the choosed items
choosed_items = []
optimal = []

# set the dataset file
file = open('./dataset/large_scale/knapPI_1_100_1000_1', 'r')
Lines = file.readlines()

count = 0
# Strips the newline character
for line in Lines:
    # Lets add a variable to count the number of line readed
    count += 1
    
    # We split the line with the space separator 
    data = line.split()

    # Read the first line of the file
    # The first line contain data about the number of items, and the knapsack capacity
    if(count == 1):
        # We get the number of item from the first line, first index
        nb_item = int(data[0])

        # We get the capacity of the knapsack from the first line second index
        capacity = int(data[1])
        continue
    
    # Sometimes the last line correspond to the line were is stored the optimal solution
    # So we check if we are reading the last line
    # And we also chack that the line is greater than to to be sure that we are reading a solution
    if count == len(Lines) and len(data) > 2: 
        # We store the solution inside the optimal array
        optimal = data
        # Its the last line of the file so we exit the loop without adding things to the choosed_items array 
        break
    else:
        # we retrieve the data from the array
        # We dont forget to cast the value 

        # First column is the value of the item
        value = int(data[0])
        # Second column is the weight of the item
        weight = int(data[1])

    # We add to the choosed_items the Item object readed
    choosed_items.append(Item(value, weight))

# We cast the values of the "optimal" array into int
optimal = list(map(int, optimal))

## 1. GENERATOR

<p>If you choosed to enter the number of the items and the capacity of the knapsack then you will need to use one of the generator presented bellow. You will be able to try different generator to see the different result given by the algorithm.</p>

##### 1. Random generator

The random generator will generate items with random values and weights. We have fixed the random state for reproducibility in the main section

In [ ]:
# create the items randomly
def random_generator():

    # generate the items randomly
    for i in range (0, nb_item):

      # Generate a weight and value randomly
      weight = random.randint(1, capacity)
      value = random.randint(1, capacity)
      
      # Create the Item and add it to the "choosed_items" array  
      choosed_items.append(Item(value, weight))

##### 2. Exponential generator

The exponential generator, generate items with a weight that follow the exponential law.

In [ ]:
def exponential_generator():

    # generate the items with exponential weight
    for i in range(0, nb_item):

      weight = int(math.exp(i))
      value = random.randint(1, capacity)
      choosed_items.append(Item(value, weight))

##### 3.Normal distribution

The normal distribution generator, generate items with a weight that follows the normal distribution.

In [ ]:
def normal_distribution_generator():
  weights = np.random.normal(capacity/2, 1, size = nb_item)
  weights = list(map(int, weights))

  # generate the items with normal distribution weight
  for i in range(0, nb_item):
    
    value = random.randint(1, capacity)
    choosed_items.append(Item(value, weights[i]))

  return weights

## 3. ALGORITHM

In this section you will find the implementation of numerous algorithm that can solve the 0-1 Knapsack problem.

##### 1. Brute Force

In [ ]:
# source : https://stackoverflow.com/questions/74174950/knapsack-problem-using-brute-force-method-in-python

# compute the total weight of a combination passed in parameter 
def compute_weight_value(combination_items):
  value, weight = 0, 0
  
  # for each item inside the combination
  for item in combination_items:
    # we compute the total weight of the combination
    weight += item.weight
    # we compute the total value of the combination
    value += item.value

  return value, weight 

# Brute force algorithm
def brute_force_solution():
  # Set the max value to 0 
  # This variable will store the maximal value finded within the generated combination
  max_value = 0
  solution = ()

  # for each generated item
  for i in range(len(choosed_items) + 1):

    # create all possible solutions
    for combination_items in itertools.combinations(choosed_items, i):
      # get total value and weight of a combination
      value, weight = compute_weight_value(combination_items) 

      # check the total weight of the combination and store only the combination that have a better value
      if value > max_value and weight <= capacity:
        # store the combination
        solution = combination_items
        # update the max_value variable
        max_value = value

  return solution

##### 2. Dynamic Programming

###### 1. Approach seen in the exercise sheet

In [ ]:
# source : https://www.askpython.com/python/examples/knapsack-problem-dynamic-programming

def dynamic_programming_solution(citems):
    table = [[0 for x in range(capacity + 1)] for x in range(nb_item + 1)] 
    for i in range(nb_item + 1): 
            for j in range(capacity + 1): 
                if i == 0 or j == 0: 
                    table[i][j] = 0
                elif citems[i-1].weight <= j: 
                    table[i][j] = max(citems[i - 1].value  + table[i - 1][j - citems[i - 1].weight],  table[i - 1][j]) 
                else: 
                    table[i][j] = table[i-1][j] 
                    
    return table[nb_item][capacity] 

###### 2. Fully polynomial time approximation scheme

In [ ]:
# source : https://www.geeksforgeeks.org/polynomial-time-approximation-scheme/

def fptas_solution():
    e = 0.5
    list = []
    citems = []
    citems = copy.deepcopy(choosed_items)
    
    for obj in choosed_items:
        list.append(obj.value)
      
    maxVal = max(list)
    k=(maxVal*e)/nb_item
    for i in range(nb_item):
        citems[i].value = math.floor(float(choosed_items[i].value)/k)

    return dynamic_programming_solution(citems)
       

###### 3. Top down approach

In [ ]:
# source : https://helloacm.com/teaching-kids-programming-0-1-knapsack-problem-via-top-down-dynamic-programming-algorithm/

from functools import cache

def dynamic_top_down_solution():

    @cache
    def dynamic_top_down_solution_rec(i, c):
        if i < 0: return 0
        
        solution = 0
        if c >= choosed_items[i].weight:
            solution = max(solution, dynamic_top_down_solution_rec(i - 1, c - choosed_items[i].weight) + choosed_items[i].value)
        
        solution = max(solution, dynamic_top_down_solution_rec(i - 1, c))
        return solution

    return dynamic_top_down_solution_rec(nb_item - 1, capacity)

##### 3. Greedy approach

###### 1. Greedy ratio

In [ ]:
# source : https://www.geeksforgeeks.org/fractional-knapsack-problem/
def greedy_ratio_solution():

    sorted_items = copy.deepcopy(choosed_items)
    sorted_items.sort(key=lambda x: (x.value/x.weight), reverse=True)
    
    initial_capacity = capacity
    finalvalue = 0.0

    # Looping through all Items
    for item in choosed_items:

        if item.weight <= initial_capacity:
            initial_capacity -= item.weight
            finalvalue += item.value

    # Returning final value
    return finalvalue

###### 2. Greedy weight

In [ ]:
def greedy_weight_solution():

    sorted_items = copy.deepcopy(choosed_items)
    sorted_items.sort(key=lambda x: x.weight, reverse=True)
    
    initial_capacity = capacity
    final_value = 0.0

    # Looping through all Items
    for item in choosed_items:

        if item.weight <= initial_capacity:
            initial_capacity -= item.weight
            final_value += item.value

    return final_value

###### 3. Greedy value

In [ ]:
def greedy_value_solution():

    sorted_items = copy.deepcopy(choosed_items)
    sorted_items.sort(key=lambda x: x.value, reverse=True)
    
    initial_capacity = capacity
    final_value = 0.0

    # Looping through all Items
    for item in sorted_items:

        if item.weight <= initial_capacity:
            initial_capacity -= item.weight
            final_value += item.value

    return final_value

###### 4. Greedy fractional

In [ ]:
def greedy_fractional_solution():

    sorted_items = copy.deepcopy(choosed_items)
    sorted_items.sort(key=lambda x: (x.value/x.weight), reverse=True)
    
    initial_capacity = capacity
    final_value = 0.0
    
    # Looping through all Items
    for item in choosed_items:

        if item.weight <= initial_capacity:
            initial_capacity -= item.weight
            final_value += item.value

        else:
            final_value += item.value * capacity / item.weight
            break

    return final_value

#### 4. Branch and bound

In [ ]:
# source : https://github.com/NateRice/0-1_Knapsack/blob/master/0-1Knapsack_keep_track_of_items.py

class Priority_Queue:
    def __init__(self):
        self.pqueue = []
        self.length = 0

    def insert(self, node, p_per_weight, sorted_items):
        for i in self.pqueue:
            get_bound(i, p_per_weight, sorted_items)
        i = 0
        while i < len(self.pqueue):
            if self.pqueue[i].bound > node.bound:
                break
            i += 1
        self.pqueue.insert(i, node)
        self.length += 1

    def print_pqueue(self):
        for i in list(range(len(self.pqueue))):
            print("pqueue", i, "=", self.pqueue[i].bound)

    def remove(self):
        try:
            result = self.pqueue.pop()
            self.length -= 1
        except:
            print("Priority queue is empty, cannot pop from empty list.")
        else:
            return result

class Node:
    def __init__(self, level, profit, weight):
        self.level = level
        self.profit = profit
        self.weight = weight
        self.items = []
        self.bound = 0


def get_bound(node, p_per_weight, sorted_items):
    if node.weight >= capacity:
        return 0
    else:
        result = node.profit
        j = node.level + 1
        totweight = node.weight
        while j <= nb_item-1 and totweight + sorted_items[j].weight <= capacity:
            totweight = totweight + sorted_items[j].weight
            result = result + choosed_items[j].value
            j += 1
        k = j
        if k <= nb_item-1:
            result = result + (capacity - totweight) * p_per_weight[k]

        return result

def branch_bound_solution():

    # initialize
    p_per_weight = []
    sorted_items = copy.deepcopy(choosed_items)

    for item in choosed_items:
        p_per_weight.append(item.value/item.weight)

    sorted_items.sort(key=lambda x: (x.value/x.weight))
    p_per_weight = sorted(p_per_weight)
    
    nodes_generated = 0
    pq = Priority_Queue()

    v = Node(-1, 0, 0) # v initialized to be the root with level = 0, profit = $0, weight = 0
    nodes_generated+=1
    max_profit = 0 # max_profit initialized to $0
    v.bound = get_bound(v, p_per_weight, sorted_items)
    pq.insert(v, p_per_weight, sorted_items)

    while pq.length != 0:
        
        v = pq.remove() #remove node with best bound
        if v.bound > max_profit: #check if node is still promising
            #set u to the child that includes the next item
            u = Node(0, 0, 0)
            nodes_generated += 1
            u.level = v.level + 1
            u.profit = v.profit + sorted_items[u.level].value
            u.weight = v.weight + sorted_items[u.level].weight
            
            #take v's list and add u's list
            u.items = v.items.copy()
            u.items.append(u.level) # adds next item

            if u.weight <= capacity and u.profit > max_profit: 
                #update max_profit
                max_profit = u.profit
                bestitems = u.items
                u.bound = get_bound(u, p_per_weight, sorted_items)
    
            if u.bound > max_profit:
                pq.insert(u, p_per_weight, sorted_items)
    
            #set u to the child that does not include the next item
            u2 = Node(u.level, v.profit, v.weight)
            nodes_generated+=1
            u2.bound = get_bound(u2, p_per_weight, sorted_items)
            u2.items = v.items.copy()
    
            if u2.bound > max_profit:
                pq.insert(u2, p_per_weight, sorted_items)
    
    return max_profit
    
    # print("\nEND maxprofit = ", maxprofit, "nodes generated = ", nodes_generated)
    # print("bestitems = ", bestitems)

#### 5. Genetic Approach

###### 1. Optimal solution

In [ ]:
# source : https://github.com/LoveBexa/applied-ai/blob/main/Genetic%20Algorithm/knapsack-genetic-algorithm-no-optima.py

############## Calculate total weight of genome ##############

# This function calculates the total weight of the entire genome
# By iterating through each item to check if it's a 1 or 0
# If it is 1 then it checks the weight in the array
# Totals this up and returns as valuez
# weight = [3, 9, 5, 6]

def calculateWeight(genome):
    total_weight = 0
    counter = 0
    # Iterate through each item in the genome
    for key, bit in enumerate(genome):
        if bit == 0:
            continue
        else:
            total_weight += choosed_items[key].weight
        counter += 1
    return total_weight

############## First generate an initial population of genome ##############

# Generates an entire population of genomes 
# By calling the genome function to randomly create a set
# Loops through genome creation until size of population met

def generateGenome(size):
    genome = []
    for number in range(size):
        # randomly generate 0 or 1 and append to array
        genome.append(random.randint(0,1))
    return genome        

############### Generate new population ##############

def generatePopulation(size):
    # Create a dict of population
    population = {}
    # Loop through size
    for number in range(size):
        # Append each generated genome to population dict
        population[number] = generateGenome(len(choosed_items))
    # Return population 
    return population
    

############### Fitness Function ##############

# Returns fitness value for each genome
# If weight is large than max_weight then return fitness value 0
# If weight is same or under max_weight then return weight as the fitness value

def checkFitness(genome):
    weight = calculateWeight(genome)
    return weight if weight <= capacity else 0


############## Selection Function ##############
# We want to pick 2 genomes randomly to battle out
# Find the winner
# Then do this again
# Then these two winners are selected! 

def selectionTourney(population):
    new_parents = {}
    counter = 0
    while counter < 2:
        # Pick genome 1 at random
        genome_one = random.randint(0, len(population)-1)
        # Pick genome 2 at random
        genome_two = random.randint(0, len(population)-1)
        # Get the fitness of each genome from the total weight (if it's over limit then its 0)
        fitness_one = checkFitness(list(population.items())[genome_one][1])
        fitness_two = checkFitness(list(population.items())[genome_two][1])        
        # Fittest genome gets added to the winners pot!
        if fitness_one > fitness_two:
            new_parents[counter] = population[genome_one]
        else: 
            new_parents[counter] = population[genome_two]
        counter += 1
        
    # Returns 2 winners as a dict
    return new_parents

############## Crossover Function ##############

# Note: to access first item in a list[0][0] to access second item in first item[0][1]
# We want to find a random point in the genome
# Cut in half at same point for both parents
# Swap 2nd half
# Produce new child! 

def crossover(parents):
    # Split parents out
    parent_one = parents[0]
    parent_two = parents[1]
    # Randomly selects an int between 2n and the length of genome 
    # I've selected items-1 as counting starts from 0 and we want total length of items in bag
    pointer = random.randint(2,len(choosed_items)-1)
    # Takes second half of genome 1 
    p1_tmp = parent_one[pointer:]
    # Takes second half of genome 2
    p2_tmp = parent_two[pointer:]
    # Takes first half of genome 1
    p1_tmp2 = parent_one[:pointer]
    # Takes first half of genome 2
    p2_tmp2 = parent_two[:pointer]
    # Makes a new genome
    p1_tmp2.extend(p2_tmp)
    p2_tmp2.extend(p1_tmp)
    # Append 2 new genomes to results
    result = []
    result.append(p1_tmp2)
    result.append(p2_tmp2)
    # Returns 2 child genomes as a new solution! 
    return result

############## Mutation Function ##############

# We want to 'mutate' aka flip a bit 
# If a problem uses 10 bits, then a good default mutation rate would be (1/10) = 0.10 or a probability of 10 percent.

def mutation(children):
    # Mutation rate should be between 0.005 - 0.01
    mutation_rate = 0.05#
    # Iterate through each bit
    for i in range(len(children)):
        for j in range(len(children[i])):
            # Generate a random number between 0 to 1 for bit 
            bit_rate = random.uniform(0,1)
            # print(mutation_rate, bit_rate)
            # If bit rate is less than mutation rate then we flip the bit
            if mutation_rate > bit_rate:
                if children[i][j] == 1:
                    children[i][j] = 0
                    # print("0", children[i][j])
                else:
                    children[i][j] = 1
                    # print("1", children[i][j])                  
    return children


############## Calculate Average Fitness and Best Fitness of a Population ##############
# This function returns the average fitness of each generation
# Also the fittest genome out of the poulation 

def fitnessStatistics(population):
    
    total_fitness = 0
    fittest_genome = [0,0,0,0]
 
    # Find the fitness of each in population
    for genome in population.values():
        # Check the fittest genome
        if checkFitness(genome) > checkFitness(fittest_genome):
            fittest_genome = genome
        # Add to total fitness
        total_fitness += checkFitness(genome)
    # ! print("Average fitness:", total_fitness/len(population))
    # ! print("Fittest Genome:", fittest_genome, "Fitness:", checkFitness(fittest_genome))
    return fittest_genome

############## Testing Station ##############

# This function generates a population the size of input amount
# Goes through GA stages
# We also want to keep generating a new population until optima solution is found
# 
def createPopulation(population, size):

    new_population = {}
    count = 0
    # Find fittest parents from tournement 
    while count < size:   
        parents = selectionTourney(population)
        #print(parents)
        # Remove fittest parents from population
        # Create cross child genomes 
        children = crossover(parents)
        #print(children)
        # Build a new population 
        mutated = mutation(children)
        # Add the new children to the dict
        new_population[count] = mutated[0]
        new_population[count+1] = mutated[1]
        count += 2
    # print(new_population)
    return new_population
    
# If we have a max generations we run through
# We can create populations to the max num of generations based on previous populations and fittest#
# With each iteratino the population should get fotter 
# We want to calculate the average fitness and the BEST fitness from each population
# And look through each population and stop the search once the local optima has been found

def genetic_optima_algorithm_solution(size, generations):
    
    # Create initial population
    next_population = generatePopulation(size)
    # ! print("Initial population:", next_population)
    
    fittest_genome = [0,0,0,0]
    
    # Initialise generation counter
    counter = 0
    found_optima = False
    while counter < generations:
        # create population from previous population
        next_population = createPopulation(next_population, size)
        # ! print("Generation",counter+1,":", next_population)
        # get the fittest genome
        fittest_genome = fitnessStatistics(next_population)
        
        # Iterate through the genomes of each population
        for genome in next_population.values():
            # print(genome)
            # If that genome is the optima
            if genome == optimal:
                # Then return this solution and break out the optima
                # ! print("Optima found after", counter+1, "generations")
                found_optima = True
                break
        if found_optima == True:
            break
        # ! print("It did not find Optima")
        counter += 1
    
    return fittest_genome

###### 2. Non Optimal solution

In [ ]:
# source : https://github.com/LoveBexa/applied-ai/blob/main/Genetic%20Algorithm/knapsack-genetic-algorithm.py

############## Calculate total weight of genome ##############

# This function calculates the total weight of the entire genome
# By iterating through each item to check if it's a 1 or 0
# If it is 1 then it checks the weight in the array
# Totals this up and returns as valuez

def calculateWeight(genome):
    total_weight = 0
    counter = 0
    # Iterate through each item in the genome
    for key, bit in enumerate(genome):
        if bit == 0:
            continue
        else:
            total_weight += choosed_items[key].weight
        counter += 1
    return total_weight

############## First generate an initial population of genome ##############

# Generates an entire population of genomes 
# By calling the genome function to randomly create a set
# Loops through genome creation until size of population met

def generateGenome(size):
    genome = []
    for number in range(size):
        # randomly generate 0 or 1 and append to array
        genome.append(random.randint(0,1))
    return genome        

############### Generate new population ##############

def generatePopulation(size):
    # Create a dict of population
    population = {}
    # Loop through size
    for number in range(size):
        # Append each generated genome to population dict
        population[number] = generateGenome(len(choosed_items))
    # Return population 
    return population
    

############### Fitness Function ##############

# Returns fitness value for each genome
# If weight is large than max_weight then return fitness value 0
# If weight is same or under max_weight then return weight as the fitness value

def checkFitness(genome):
    weight = calculateWeight(genome)
    return weight if weight <= capacity else 0


############## Selection Function ##############
# We want to pick 2 genomes randomly to battle out
# Find the winner
# Then do this again
# Then these two winners are selected! 

def selectionTourney(population):
    new_parents = {}
    counter = 0
    while counter < 2:
        # Pick genome 1 at random
        genome_one = random.randint(0, len(population)-1)
        # Pick genome 2 at random
        genome_two = random.randint(0, len(population)-1)
        # Get the fitness of each genome from the total weight (if it's over limit then its 0)
        fitness_one = checkFitness(list(population.items())[genome_one][1])
        fitness_two = checkFitness(list(population.items())[genome_two][1])        
        # Fittest genome gets added to the winners pot!
        if fitness_one > fitness_two:
            new_parents[counter] = population[genome_one]
        else: 
            new_parents[counter] = population[genome_two]
        counter += 1
        
    # Returns 2 winners as a dict
    return new_parents

############## Crossover Function ##############

# Note: to access first item in a list[0][0] to access second item in first item[0][1]
# We want to find a random point in the genome
# Cut in half at same point for both parents
# Swap 2nd half
# Produce new child! 

def crossover(parents):
    # Split parents out
    parent_one = parents[0]
    parent_two = parents[1]
    # Randomly selects an int between 2n and the length of genome 
    # I've selected items-1 as counting starts from 0 and we want total length of items in bag
    pointer = random.randint(2,len(choosed_items)-1)
    # Takes second half of genome 1 
    p1_tmp = parent_one[pointer:]
    # Takes second half of genome 2
    p2_tmp = parent_two[pointer:]
    # Takes first half of genome 1
    p1_tmp2 = parent_one[:pointer]
    # Takes first half of genome 2
    p2_tmp2 = parent_two[:pointer]
    # Makes a new genome
    p1_tmp2.extend(p2_tmp)
    p2_tmp2.extend(p1_tmp)
    # Append 2 new genomes to results
    result = []
    result.append(p1_tmp2)
    result.append(p2_tmp2)
    # Returns 2 child genomes as a new solution! 
    return result

############## Mutation Function ##############

# We want to 'mutate' aka flip a bit 
# If a problem uses 10 bits, then a good default mutation rate would be (1/10) = 0.10 or a probability of 10 percent.

def mutation(children):
    # Mutation rate should be between 0.005 - 0.01
    mutation_rate = 0.05#
    # Iterate through each bit
    for i in range(len(children)):
        for j in range(len(children[i])):
            # Generate a random number between 0 to 1 for bit 
            bit_rate = random.uniform(0,1)
            # print(mutation_rate, bit_rate)
            # If bit rate is less than mutation rate then we flip the bit
            if mutation_rate > bit_rate:
                if children[i][j] == 1:
                    children[i][j] = 0
                    # print("0", children[i][j])
                else:
                    children[i][j] = 1
                    # print("1", children[i][j])                  
    return children


############## Calculate Average Fitness and Best Fitness of a Population ##############
# This function returns the average fitness of each generation
# Also the fittest genome out of the poulation 

def fitnessStatistics(population):
    
    total_fitness = 0
    fittest_genome = [0,0,0,0]
 
    # Find the fitness of each in population
    for genome in population.values():
        # Check the fittest genome
        if checkFitness(genome) > checkFitness(fittest_genome):
            fittest_genome = genome
        # Add to total fitness
        total_fitness += checkFitness(genome)
    # print("Average fitness:", total_fitness/len(population))
    # print("Fittest Genome:", fittest_genome, "Fitness:", checkFitness(fittest_genome))
    return fittest_genome
############## Testing Station ##############

# This function generates a population the size of input amount
# Goes through GA stages
# We also want to keep generating a new population until optima solution is found
# 
def createPopulation(population, size):
    
    new_population = {}
    count = 0
    # Find fittest parents from tournement 
    while count < size:   
        parents = selectionTourney(population)
        #print(parents)
        # Remove fittest parents from population
        # Create cross child genomes 
        children = crossover(parents)
        #print(children)
        # Build a new population 
        mutated = mutation(children)
        # Add the new children to the dict
        new_population[count] = mutated[0]
        new_population[count+1] = mutated[1]
        count += 2
    # print(new_population)
    return new_population
    
# If we have a max generations we run through
# We can create populations to the max num of generations based on previous populations and fittest#
# With each iteratino the population should get fotter 
# We want to calculate the average fitness and the BEST fitness from each population
# And look through each population and stop the search once the local optima has been found

def genetic_no_optima_algorithm_solution(size, generations):
    
    # Create initial population
    next_population = generatePopulation(size)
    # ! print("Initial population:", next_population)

    fittest_genome = 0
    
    # Initialise generation counter
    counter = 0
    found_optima = False
    while counter < generations:
        # create population from previous population
        next_population = createPopulation(next_population, size)
        # ! print("Generation",counter+1,":", next_population)
        fittest_genome = fitnessStatistics(next_population)
        # Iterate through the genomes of each population
        for genome in next_population.values():
            # print(genome)
            # If that genome is the optima
            if checkFitness(genome) == capacity:
                # Then return this solution and break out the optima
                # ! print("Optima found after", counter+1, "generations")
                found_optima = True
                break
        if found_optima == True:
            break
        # ! print("It did not find Optima")
        counter += 1
    
    return fittest_genome

#### 6. Randomized approach

In [ ]:
def cmcm_solution():
    vbx=0
    x=buckets = [0] * nb_item

    bestx = [0] * nb_item

    for i in range(nb_item):
      # print("x",x)
      # print("bestx",bestx)

      k= random.randint(0, nb_item - 1)

      # print("the value of =============== k",k)
      # print("VBX ",vbx)

      if x[k]==0:
        x[k]=1
      else:
        x[k]=0
      newx=x
      # print("newx",newx)
      cc=0
      for t in range(nb_item):
        if newx[t]==1:
          cc=cc+choosed_items[t].weight
      # print("total c",cc)

      if cc<=capacity:
        x=newx
        # print("x",x)
      
        vx=0
        for r in range(nb_item):
          if x[r]==1:
            vx=vx+choosed_items[r].value 
        # print("vx",vx) 

        # print("vbx befor",vbx)   
        pp=vbx
      
          
        vbx=0
        for r2 in range(nb_item):
            if bestx[r2]==1:
              vbx=vbx+choosed_items[r2].value
              # print("vbx- pp",vbx)    


        if vx>vbx:
          for i in range(nb_item):
            bestx[i]=x[i]
            vbx=vx
          
          # print("bestx comparaison avec x",bestx)
      
    return vbx

## 2. TEST

In this section you will find the call of the algorithm implemented in the section Algorithm

### Clear Choosed Items

If you want to clear the "choosed_items" array without re-starting all the project, just launch this block.

In [ ]:
# clear the choosed items array
choosed_items = []

In [ ]:
def draw_graph():
    sorted_items = choosed_items
    # sorted_items.sort(key=lambda x: x.weight)

    x, y = [], []
    for i in range (0, len(sorted_items)):
        x.append(i)
        y.append(sorted_items[i].weight)
        
    # plotting the points
    plt.plot(x, y)

    # naming the x axis
    plt.xlabel('x - items')
    # naming the y axis
    plt.ylabel('y - weight')

    # function to show the plot
    plt.show()

### 1. Generator

If you are not using a dataset but have entered the data from the input, choose the type of generator you want the items to be generated with.

#### 1. Random

In [ ]:
random_generator()

#### 2. Exponential

In [ ]:
exponential_generator()

#### 3. Normal distribution

In [ ]:
normal = normal_distribution_generator()
# plt.hist(normale, 50)
# plt.show()

### 3. Brute force

In [ ]:
def brute_force_test():
    # Start the tracking of the memory usage
    tracemalloc.start()
    # get the starting time
    begin = time.time()
    solution = brute_force_solution()
    # get the time of the end of the computation 
    end = time.time()
    # get the memory usage
    mem_usage, mem_peak = tracemalloc.get_traced_memory()
    # stop the tracemalloc
    tracemalloc.stop()
    
    # Clear traces of memory blocks allocated by Python
    tracemalloc.clear_traces()
    # Set the peak size of memory blocks traced by tracemalloc to the current size
    tracemalloc.reset_peak()

    # Save the solution in the solutions array
    problem_solutions.append(Solution((end - begin), BRUTE_FORCE, compute_total_value(solution), mem_usage * 0.000001, mem_peak * 0.000001))

    # display the data obtainer from the computation
    print("brute force = %s secondes" % problem_solutions[len(problem_solutions) - 1].time) 
    print(f"Memory usage: {mem_usage * 0.000001}MB; Peak: {mem_peak * 0.000001}MB")
    print("solution brute force = " + str(problem_solutions[len(problem_solutions) - 1].solution))
    print("---")

brute_force_test()

### 4. Dynamic programming

#### 1. exercise sheet

In [ ]:
def dynamic_programming_test():
    
    # Start the tracking of the memory usage
    tracemalloc.start()
    # get the starting time
    begin = time.time()
    solution = dynamic_programming_solution(choosed_items)
    # get the time of the end of the computation
    end = time.time()
    # get the memory usage
    mem_usage, mem_peak = tracemalloc.get_traced_memory()
    # stop the tracemalloc
    tracemalloc.stop()
    
    # Clear traces of memory blocks allocated by Python
    tracemalloc.clear_traces()
    # Set the peak size of memory blocks traced by tracemalloc to the current size
    tracemalloc.reset_peak()
    
    # calculate the duration of the algorithm
    duration = (end - begin)
    # Save the solution in the solutions array
    problem_solutions.append(Solution(duration, DYNAMIC_PROG, solution, mem_usage * 0.000001, mem_peak * 0.000001))

    # display the data obtainer from the computation
    print("dynamic = %s secondes" % problem_solutions[len(problem_solutions) - 1].time) 
    print(f"Memory usage: {mem_usage * 0.000001}MB; Peak: {mem_peak * 0.000001}MB")
    print("solution = " + str(problem_solutions[len(problem_solutions) - 1].solution))
    print("---")
    
dynamic_programming_test()

#### 2. FPTAS

In [ ]:
def fptas_test():

    # Start the tracking of the memory usage
    tracemalloc.start()
    # get the starting time
    begin = time.time()
    solution = fptas_solution()
    # get the time of the end of the computation
    end = time.time()
    # get the memory usage
    mem_usage, mem_peak = tracemalloc.get_traced_memory()
    # stop the tracemalloc
    tracemalloc.stop()
    
    # Clear traces of memory blocks allocated by Python
    tracemalloc.clear_traces()
    # Set the peak size of memory blocks traced by tracemalloc to the current size
    tracemalloc.reset_peak()

    # calculate the duration of the algorithm
    duration = (end - begin)
    # Save the solution in the solutions array
    problem_solutions.append(Solution(duration, FPTAS, solution, mem_usage * 0.000001, mem_peak * 0.000001))

    # display the data obtainer from the computation
    print("fptas = %s secondes" % problem_solutions[len(problem_solutions) - 1].time) 
    print(f"Memory usage: {mem_usage * 0.000001}MB; Peak: {mem_peak * 0.000001}MB")
    print("solution = " + str(problem_solutions[len(problem_solutions) - 1].solution))
    print("---")

fptas_test()

#### 3. top down approach

In [ ]:
def dynamic_top_down_test():

    # Start the tracking of the memory usage
    tracemalloc.start()
    # get the starting time
    begin = time.time()
    solution = dynamic_top_down_solution()
    # get the time of the end of the computation
    end = time.time()
    # get the memory usage
    mem_usage, mem_peak = tracemalloc.get_traced_memory()
    # stop the tracemalloc
    tracemalloc.stop()

    # Clear traces of memory blocks allocated by Python
    tracemalloc.clear_traces()
    # Set the peak size of memory blocks traced by tracemalloc to the current size
    tracemalloc.reset_peak()

    # calculate the duration of the algorithm
    duration = (end - begin)
    # Save the solution in the solutions array
    problem_solutions.append(Solution(duration, DYNAMIC_PROG_TOP_DOWN, solution, mem_usage * 0.000001, mem_peak * 0.000001))

    print("dynamic top-down = %s secondes" % problem_solutions[len(problem_solutions) - 1].time)
    print(f"Memory usage: {mem_usage * 0.000001}MB; Peak: {mem_peak * 0.000001}MB")
    print("solution = " + str(problem_solutions[len(problem_solutions) - 1].solution))
    print("---")
    
dynamic_top_down_test()

### 5. Greedy solution

#### 1. value solution

In [ ]:
def greedy_value_test():
    
    # Start the tracking of the memory usage
    tracemalloc.start()
    # get the starting time
    begin = time.time()
    solution = greedy_value_solution()
    # get the time of the end of the computation
    end = time.time()
    # get the memory usage
    mem_usage, mem_peak = tracemalloc.get_traced_memory()
    # stop the tracemalloc
    tracemalloc.stop()

    # Clear traces of memory blocks allocated by Python
    tracemalloc.clear_traces()
    # Set the peak size of memory blocks traced by tracemalloc to the current size
    tracemalloc.reset_peak()

    # calculate the duration of the algorithm
    duration = (end - begin)
    # Save the solution in the solutions array
    problem_solutions.append(Solution(duration, GREEDY_VALUE, solution, mem_usage * 0.000001, mem_peak * 0.000001))

    print("greedy value = %s secondes" % problem_solutions[len(problem_solutions) - 1].time)
    print(f"Memory usage: {mem_usage * 0.000001}MB; Peak: {mem_peak * 0.000001}MB")
    print("solution = " + str(problem_solutions[len(problem_solutions) - 1].solution))
    print("---")

greedy_value_test()

#### 2. weight solution

In [ ]:
def greedy_weight_test():
    
    # Start the tracking of the memory usage
    tracemalloc.start()
    # get the starting time
    begin = time.time()
    solution = greedy_weight_solution()
    # get the time of the end of the computation
    end = time.time()
    # get the memory usage
    mem_usage, mem_peak = tracemalloc.get_traced_memory()
    # stop the tracemalloc
    tracemalloc.stop()

    # Clear traces of memory blocks allocated by Python
    tracemalloc.clear_traces()
    # Set the peak size of memory blocks traced by tracemalloc to the current size
    tracemalloc.reset_peak()

    # calculate the duration of the algorithm
    duration = (end - begin)
    # Save the solution in the solutions array
    problem_solutions.append(Solution(duration, GREEDY_VALUE, solution, mem_usage * 0.000001, mem_peak * 0.000001))

    print("greedy weight = %s secondes" % problem_solutions[len(problem_solutions) - 1].time)
    print(f"Memory usage: {mem_usage * 0.000001}MB; Peak: {mem_peak * 0.000001}MB")
    print("solution = " + str(problem_solutions[len(problem_solutions) - 1].solution))
    print("---")

greedy_weight_test()

#### 3. ratio solution

In [ ]:
def greedy_ratio_test():
    
    # Start the tracking of the memory usage
    tracemalloc.start()
    # get the starting time
    begin = time.time()
    solution = greedy_ratio_solution()
    # get the time of the end of the computation
    end = time.time()
    # get the memory usage
    mem_usage, mem_peak = tracemalloc.get_traced_memory()
    # stop the tracemalloc
    tracemalloc.stop()

    # Clear traces of memory blocks allocated by Python
    tracemalloc.clear_traces()
    # Set the peak size of memory blocks traced by tracemalloc to the current size
    tracemalloc.reset_peak()

    # calculate the duration of the algorithm
    duration = (end - begin)
    # Save the solution in the solutions array
    problem_solutions.append(Solution(duration, GREEDY_RATIO, solution, mem_usage * 0.000001, mem_peak * 0.000001))

    print("greedy ratio = %s secondes" % problem_solutions[len(problem_solutions) - 1].time)
    print(f"Memory usage: {mem_usage * 0.000001}MB; Peak: {mem_peak * 0.000001}MB")
    print("solution = " + str(problem_solutions[len(problem_solutions) - 1].solution))
    print("---")

greedy_ratio_test()

#### 3. Fractional solution

In [ ]:
def greedy_fractional_test():

    # Start the tracking of the memory usage
    tracemalloc.start()
    # get the starting time
    begin = time.time()
    solution = greedy_fractional_solution()
    # get the time of the end of the computation
    end = time.time()
    # get the memory usage
    mem_usage, mem_peak = tracemalloc.get_traced_memory()
    # stop the tracemalloc
    tracemalloc.stop()

    # Clear traces of memory blocks allocated by Python
    tracemalloc.clear_traces()
    # Set the peak size of memory blocks traced by tracemalloc to the current size
    tracemalloc.reset_peak()

    # calculate the duration of the algorithm
    duration = (end - begin)
    # Save the solution in the solutions array
    problem_solutions.append(Solution(duration, GREEDY_FRACTIONAL, solution, mem_usage * 0.000001, mem_peak * 0.000001))

    print("greedy fractional = %s secondes" % problem_solutions[len(problem_solutions) - 1].time)
    print(f"Memory usage: {mem_usage * 0.000001}MB; Peak: {mem_peak * 0.000001}MB")
    print("solution = " + str(problem_solutions[len(problem_solutions) - 1].solution))
    print("---")

greedy_fractional_test()

### 6. Branch & bound

In [ ]:
def branch_bound_test():

    # Start the tracking of the memory usage
    tracemalloc.start()
    # get the starting time
    begin = time.time()
    solution = branch_bound_solution()
    # get the time of the end of the computation
    end = time.time()
    # get the memory usage
    mem_usage, mem_peak = tracemalloc.get_traced_memory()
    # stop the tracemalloc
    tracemalloc.stop()

    # Clear traces of memory blocks allocated by Python
    tracemalloc.clear_traces()
    # Set the peak size of memory blocks traced by tracemalloc to the current size
    tracemalloc.reset_peak()

    # calculate the duration of the algorithm
    duration = (end - begin)
    # Save the solution in the solutions array
    problem_solutions.append(Solution(duration, BRANCH_BOUND, solution, mem_usage * 0.000001, mem_peak * 0.000001))

    print("branch bound = %s secondes" % problem_solutions[len(problem_solutions) - 1].time)
    print(f"Memory usage: {mem_usage * 0.000001}MB; Peak: {mem_peak * 0.000001}MB")

    print("solution = " + str(problem_solutions[len(problem_solutions) - 1].solution))
    print("---")

branch_bound_test()

### 7. Genetic

#### 1. Optimal solution

In [ ]:
def genetic_optima_algorithm_test():
    
    # Start the tracking of the memory usage
    tracemalloc.start()
    # get the starting time
    begin = time.time()
    
    ############## Checking through each weight ##############
    # First number is total genomes in population
    # Second number is the number of generations we want to create
    ##########################################################
    fittest_genome = genetic_optima_algorithm_solution(10, 100) 
    
    # get the time of the end of the computation
    end = time.time()
    # get the memory usage
    mem_usage, mem_peak = tracemalloc.get_traced_memory()
    # stop the tracemalloc
    tracemalloc.stop()

    # Clear traces of memory blocks allocated by Python
    tracemalloc.clear_traces()
    # Set the peak size of memory blocks traced by tracemalloc to the current size
    tracemalloc.reset_peak()

    # calculate the duration of the algorithm
    duration = (end - begin)
    # compute the solution from a genome
    solution = compute_genome_solution(fittest_genome)

    # Save the solution in the solutions array
    problem_solutions.append(Solution(duration, GENETIC, solution, mem_usage * 0.000001, mem_peak * 0.000001))

    print("genetic = %s secondes" % problem_solutions[len(problem_solutions) - 1].time)
    print(f"Memory usage: {mem_usage * 0.000001}MB; Peak: {mem_peak * 0.000001}MB")
    print("solution = " + str(problem_solutions[len(problem_solutions) - 1].solution))
    print("---")

genetic_optima_algorithm_test()

#### 2. No optimal solution

In [ ]:
def genetic_no_optima_algorithm_test():
    
    # Start the tracking of the memory usage
    tracemalloc.start()
    # get the starting time
    begin = time.time()

    ############## Checking through each weight ##############
    # First number is total genomes in population
    # Second number is the number of generations we want to create
    ##########################################################
    fittest_genome = genetic_no_optima_algorithm_solution(100, 1000)
    
    # get the time of the end of the computation
    end = time.time()
    # get the memory usage
    mem_usage, mem_peak = tracemalloc.get_traced_memory()
    # stop the tracemalloc
    tracemalloc.stop()

    # Clear traces of memory blocks allocated by Python
    tracemalloc.clear_traces()
    # Set the peak size of memory blocks traced by tracemalloc to the current size
    tracemalloc.reset_peak()

    # calculate the duration of the algorithm
    duration = (end - begin)
    # compute the solution from a genome
    solution = compute_genome_solution(fittest_genome)

    # Save the solution in the solutions array
    problem_solutions.append(Solution(duration, GENETIC_NO_OPTIMA, solution, mem_usage * 0.000001, mem_peak * 0.000001))

    print("genetic no optima = %s secondes" % problem_solutions[len(problem_solutions) - 1].time)
    print(f"Memory usage: {mem_usage * 0.000001}MB; Peak: {mem_peak * 0.000001}MB")

    print("solution = " + str(problem_solutions[len(problem_solutions) - 1].solution))
    print("---")

genetic_no_optima_algorithm_test()

### 8. CMCM

In [ ]:
def cmcm_test():

    # Start the tracking of the memory usage
    tracemalloc.start()
    # get the starting time
    begin = time.time()
    solution = cmcm_solution()
    # get the time of the end of the computation
    end = time.time()
    # get the memory usage
    mem_usage, mem_peak = tracemalloc.get_traced_memory()
    # stop the tracemalloc
    tracemalloc.stop()

    # Clear traces of memory blocks allocated by Python
    tracemalloc.clear_traces()
    # Set the peak size of memory blocks traced by tracemalloc to the current size
    tracemalloc.reset_peak()

    # calculate the duration of the algorithm
    duration = (end - begin)
    # Save the solution in the solutions array
    problem_solutions.append(Solution(duration, RANDOMIZED, solution, mem_usage * 0.000001, mem_peak * 0.000001))

    print("radomized = %s secondes" % problem_solutions[len(problem_solutions) - 1].time)
    print(f"Memory usage: {mem_usage * 0.000001}MB; Peak: {mem_peak * 0.000001}MB")
    print("solution = " + str(problem_solutions[len(problem_solutions) - 1].solution))
    print("---")

cmcm_test()

### Information about the problem

In [ ]:
#####  Display the date of the problem #####
print("Knapsack capacity : " + str(capacity))
print("Number of items : " + str(nb_item) + "\n")

# get the optimal solution from a the genome (when working with the dataset)
omptimum = compute_genome_solution(optimal)

# display optimal solution (when working with the dataset)
print("Optimal solution : " + str(omptimum))
print("Optimal genome: " + str(optimal) + "\n")

# display the choosed items
Item.print_items(choosed_items)

### Display Solutions

Launching this block after the Test section will display all the solution of each algorithm launched, with their time of execution, the solution finded and even the memory used during the computation

In [ ]:
# Create a table to display the solution
table = PrettyTable(["Algo", "Time", "Mem usage", "Mem peak", "Solution"])

# display the all the solutions
for solution in (problem_solutions):
    table.add_row([solution.name, str(solution.time), str(solution.mem_usage/4) + " MB", str(solution.mem_peak/4) + " MB", solution.solution])

print(table)

### Clear Solutions

If you want to clear the "problem_solutions" array without re-starting all the project, just launch this block.

In [ ]:
# clear the solutions from the array
problem_solutions = []